# Assignment 3 - Step 1: Data collection
Group 11:
- Lisa Driessen - r0675727
- Laura Fernández López - r0877908
- Silvia María Goñi Mendia - r0877434
- Peter Day - r0866276

In [1]:
import threading
import os, re
import pandas as pd
from operator import itemgetter
from pyspark.streaming import StreamingContext

## Collect data

In [2]:
# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
ssc = StreamingContext(sc, 60)

In [6]:
lines = ssc.socketTextStream("localhost", 8080)

# Write path for saving folder, "nt" for Windows, "posix" for Mac and Linux
if (os.name == "nt"):
    lines.repartition(1).saveAsTextFiles("C:/Users/peter/Documents/spark")
elif (os.name == "posix"):
    lines.repartition(1).saveAsTextFiles("/Users/silvia/Desktop/guardar/")

In [7]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [8]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


## Save data as csv file

In [10]:
# Write same path as in the previous step
if (os.name == "nt"):
    rootPath = "C:/Users/peter/Documents/spark"
elif (os.name == "posix"):
    rootPath = "/Users/silvia/Desktop/guardar/"

# Only read files called "part-00000", ignoring "_SUCCESS" files
fileType = 'part-00000'
notInclude = 'crc'
files = []
directories = []
messages = []

# Create dataframe to include all messages
df = pd.DataFrame()
dfAll = pd.DataFrame()
print(dfAll.shape[0])
channel1 = 'loltyler1'
channel2 = 'jinnytty'

i=0
for dirpath, dirnames, filenames in os.walk(rootPath):
    for file in os.scandir(dirpath):
        if not('-' in file.path):
            continue
        else:
            if file.is_file():
                if (str(file).find(fileType) != -1) and (str(file).find('crc') == -1):
                    df = pd.DataFrame(pd.read_json(file, orient='records', lines=True, encoding = 'ISO-8859-1'))
                    df.drop(df.columns[[0]], axis = 1, inplace = True)
                    dfAll = pd.concat([dfAll, df])
                    print(dfAll.shape)

# Save df as csv file
dfAll.to_csv("messages.csv", index=False)

0
(1, 3)
(135, 3)
(280, 3)
(416, 3)
(513, 3)
